**research_work_log.ipynb - research work log**

# working files

analysis:

- network data creation: [analysis-network_data_output-GRP.ipynb](./analysis/network_data/analysis-network_data_output-GRP.ipynb)
- network analysis with R: [R_network_analysis_test.R.ipynb](./analysis/network_data/R_network_analysis_test.R.ipynb)

data prep:

- data exploration notebook: [analysis-data_exploration.ipynb](./analysis/analysis-data_exploration.ipynb)
- network data creation example notebook: [analysis-network_data_output_example.ipynb](./analysis/analysis-network_data_output_example.ipynb)
- `Article` overview notebook: [analysis-article_overview.ipynb](./analysis/analysis-article_overview.ipynb)
- `Article_Data` overview notebook: [analysis-article_data_overview.ipynb](./analysis/analysis-article_data_overview.ipynb)
- single-word names notebook: [analysis-omit_single_names.ipynb](./analysis/analysis-omit_single_names.ipynb)

    - includes loops that output updates on long-run processing every X iterations.
    
- updating unit test fixtures for testing network data output: [test_data-update_data-network_output.ipynb](./data/test_data/test_data-update_data-network_output.ipynb)


# ====> TODO

# analysis

## CURRENT - analysis data creation

### Create network time slices

Notes on storage:

- for each unique timeslice, need a unique identifier I can use to identify derived data for each time slice.

    - could be label
    
        - perhaps pattern: `<type>-<paper>-<coder>-<start_date>-<end_date>`
        - examples:
        
            - `week-grp-automated-20050501-20050507`
            - `7day-grp-automated-20050502-20050508`
    
        - type would be either:
        
            - actual time period:
            
                - week
                - month
                - quarter
                - half-year
                - year
            
            - conceptual time period:
            
                - sliding week = "7day"
                - sliding month = "31day"
                - sliding quarter = "92day"
                - sliding half-year = "183day"
                - sliding year = "365day"
    
    - might use foreign key directly to output log rows...
    
- need to add columns to `NetworkDataOutputLog`?

    - coder ID list
    - automated coder type
    - earliest article date
    - latest article date

#### network snapshots for actual weeks, months, etc.

TODO:

- // make the logging of data output store the spec first, then update the record with data once the output is complete.
- memory management making network data - see if I can get glassbox to render (give 32 GB to VM).

To start, need to create network time slices tied to actual date-time periods for GRP local hard-news articles:

- network for every complete calendar week.
- network for every complete calendar month.
- network for every complete calendar quarter.
- network for every complete calendar year.


#### sliding time period network snapshots

Then, need to do sliding time periods starting at first day with articles, through to the last date where there is a complete time period.

For argument's sake, the time periods and their duration in days:

- week = 7
- month = 31
- quarter = 92 (round up 365/4)
- half-year = 183 (round up 365/2)
- year = 365

### Metrics within each time slice

Within any given timeslice, need to be able to compute:

- mean/median/mode degree of all reporters
- mean/median/mode degree of all sources
- per person:

    - degree
    - centrality
    - average tie weight

- per author-source pair:

    - count of articles shared within time period.
    - 
    
- per source-source pair (co-occurring in an article):

    - count of articles shared within time period.

And, need to figure out how to store all this - probably need a database table or two, maybe in `context_analysis` rather than `context_text`.

## time slice analysis

### pairwise graph correlations - actual time periods

pairwise correlations between pairs of same actual units of time across entire date range (1 & 2; 2 & 3, 3 & 4; etc.):

- week (will be low, want to see pattern)
- month
- quarter
- half-year
- year

Once you have series of correlations, look at time series, average correlation, standard deviation, etc., so you have an idea of how any given period-to-period similarity compares to the baseline/average.

### pairwise graph correlations - sliding time periods

pairwise correlations between pairs of same actual units of time across entire date range (1 & 2; 2 & 3, 3 & 4; etc.):

- 7-day (week)
- 31-day (month)
- 92-day (quarter)
- 183-day (half-year)
- 365-day (year)

Here, you'd start with the earliest date, and then for a give duration, correlate the time period starting the the current date to the time period that starts directly after the first one. Example of 7days snapshots (weeks-ish):

- for each date between first and last date of articles in a data set until there are no more full "next period"s:

    - retrieve 7days snapshot for current date.
    - retrieve next 7days snapshot - 7days snapshot starting on current date + 7 days.
    - correlate and store correlation.
    
Once you have series of correlations, look at time series, average correlation, standard deviation, etc., so you have an idea of how any given period-to-period similarity compares to the baseline/average.

### TODO

- correlation patterns

    - weeks:

        - week to week across all weeks
        - week to containing month
        - week to containing quarter
        - week to containing year

    - months:

        - month to month across all months
        - month to containing quarter
        - month to containing year

    - quarters

        - quarter to quarter across all quarters
        - quarter to containing year

    - years

        - year-to-year

- consider this for other time slices, also - 6 months, 3 months, 4 weeks, etc.
- do within-year correlations for every set of 365 days.
- look at correlation patterns of year-before and year-after for all dates that have a year before and after.
- modeling:

    - input variables:
    
        - article count before
        - average sources per article before
        - source count per reporter before
        - co-author by reporter before
        - average source count of co-authors by reporter before
        
    - outcome variables (use all inputs to predict each):
    
        - article count after
        - average sources per article after
        - source count per reporter after
        - co-author by reporter after
        - average source count of co-authors by reporter after

## structural equivalence

Hmmm.

## Models

### Model 1 - source use influence on subsequent use

How strong is influence of having used a source in time period 0 on using that source again (where you could look at two or three horizons – 6 months, a year, 5 years)?  This is a diagnostic as much as anything – you’d expect it to be a positive predictor.

- unit of analysis is author-source pairs
- for every author-source pair where they exist in a given period, look at how that predicts use in subsequent period.
- might also have a variable for “ever used” and/or “previous use count”, and then “used in last time period”, to predict use in subsequent time period.
- Also could look at skipping periods. Quarters for example:

    - start with each quarter predicting a subsequent quarter.


## Model 2 - do sources tend to be used together?

Model 2: If sources appear within articles together, does use of one by a reporter influence that reporter to then use others of those co-sources?  In theory, sources influence reporters as they interact over time and the context of some stories can also become a journalistic heuristic for source selection (the usual sources for a particular context).  These influences on a reporter might manifest in quoting more sources that appear in articles with people who the journalist quotes.  This model tests:

- If sources are together in an article in time period 0…
- And a reporter uses one of those sources in time period 1…
- Is that reporter more likely to use other sources who appeared with that source in the same time period or in a subsequent time period (period 2)?
- Does the weight of “together in article” tie between sources matter (so if they only appear together once, versus appearing together many times across many articles).
- Variable details

    - Variables for each reporter-source pair:

        - At time 0:
        
            - Quoted in time 0?
        
        - At time 1:

            - Quoted in time 1?
            - Co-located in publication with any source of the reporter in time 0?  (Boolean)
            - Count of publications in which this source was co-located with other sources quoted by this reporter in time 0.
            - Count of this reporter’s time 0 (and time 1?) sources this source had been co-located with.

# ====> DEFERRED

# coding

- notebooks:

    - OpenCalais V2 (and tagging local hard news):
        
        - [article_coding.ipynb](./data/article_coding/article_coding.ipynb)
        - [newsbank-article_coding.ipynb](./data/article_coding/newsbank-article_coding.ipynb)

- code the Detroit News articles.

    - for work needed for author strings to be able to find local news correctly, see:
    
        - `context_text/collectors/newsbank/newspapers/DTNB.py`
        - notebook with code to start to deal with this: [data_creation-filter_locally_implemented_hard_news.ipynb](./methods/data_creation/data_creation-filter_locally_implemented_hard_news.ipynb)
    
    - for filtering down to local news:
    
        - notes: [newsbank-article_coding.ipynb](./data/article_coding/newsbank-article_coding.ipynb)
        - Business, Metro, and Nation
        - local byline

- code to filter out those with single names... where?

    - exploration
    
- use tags to programatically filter and bin sets of articles I am interested in, then just use those tags to generate network data in [Network Builder](https://research.local/research/context/text/output/network).

    - remove single names (add this to the processing in network builder?).
    - only appropriate sections
    - year before and after layoffs.
    - every 3 months of the year before and after layoffs.
    - every month of the year before and after layoffs.
    - every week of the year before and after layoffs.
    


# Performance

- look at queries being run, make some indexes (also for context-proper).
- build an outputter for node list plus edge list.
- try out the python graph thing.

# ====> DONE

- _NOTE: most recently completed is at the bottom._

# Dissertation proposal

- Back to [Table of Contents](Table-of-Contents)

DONE:

- lay out the Journalism-centric thing i’ve been working on.

    - start the outline I already wrote, see how it looks.

- list of modules I have been preparing for re: dissertation, including all work needed for each, which parts are completed, and a rough estimate timeline for work needed to be done, broken out by task.
- a brief dissertation outline based on the work I have already completed, with three papers broken out into details of each.
- For the three papers, integrate module estimates with additional work needed for each, which parts are completed, and a rough estimate timeline for work needed to be done, broken out by task.
- a narrative for each paper (will try to get into design and methods, including picking data from sets of data I have access to).  For the methods paper, i’ll have a draft completed.


# analysis

## data prep

### programmatic network data creation

- code:

    - template: `templates/context_text/context_text_output_network.html`
    - view function: `views.py --> output_network()`

        - forms:

            - `forms.ArticleSelectForm` - "Select Articles" section of the page.
            - `forms.NetworkOutputForm` - bottom of "Configure Network" section of the page.
            - `forms.PersonSelectForm` - "Select People" section of the page.
            - `forms.RelationSelectionForm` - top of "Configure Network section of the page (things that start with "relation-*").

    - view function calls `NetworkOutput.process_network_output_request( request_IN, debug_flag_IN )` for actual processing. `process_network_output_request`:

        - can also be called with `params_IN`, effectively storing the same parameters that come in the POST inputs for a POST request in a dictionary that is passed as `params_IN`.

        - calls `NetworkOutput.create_network_query_set()` to create QuerySet of Article_Data from parameters passed in.

            - calls `NetworkOutput.create_query_set()`. This is where all the real work happens.
            
                - when it retrieves `Article_Author`s and `Article_Subject`s, calls `Article_Data` methods `get_article_authors_qs` and `get_quoted_article_sources_qs`. These are the methods that now can filter to exclude tags, and to exclude single names (based on spaces in `verbatim_name`).

        - calls `NetworkOutput.render_network_data()` to take QuerySet from above method and turn it into network data.

            - calls `NetworkOutput.create_person_dict()`.

                - calls `NetworkOutput.create_person_query_set()`. Depending on how you want to pull in people (effectively, people decides the number of rows/columns, and who is represented in the matrix of network data):

                    - "all" - build dictionary including all people in every `Article_Data` instance, bar none.
                    - "articles" (and default) - use `NetworkOutput.create_network_query_set()` to build person set from same `Article_Data`'s selected for creating network data.
                    - "custom" - call `NetworkOutput.create_query_set( NetworkOutput.PARAM_PERSON_PREFIX )` to build separate QuerySet from parameters passed in to pull in needed people.

            - calls `NetworkOutput.get_NDO_instance()` to get NetworkDataOutput child render class.
            - With NetworkDataOutput class:

                - `set_query_set()` - stores Article_Data queryset.
                - `set_person_dictionary()` - stores person dictionary (set of people who will be rows and columns in network data matrix).
                - `initialize_from_params()` - passes params to NetworkDataOutput instance, for any other information it needs.
                - calls `render()` method on NetworkDataOutput class instance to render.

                    - when it retrieves `Article_Author`s and `Article_Subject`s, calls `Article_Data` methods `get_article_authors_qs` and `get_quoted_article_sources_qs`. These are the methods that now can filter to exclude tags, and to exclude single names (based on spaces in `verbatim_name`).

#### Create network data progammatically with JSON

- look at [Network Builder](https://research.local/research/context/text/output/network) - see how hard to make it callable with a JSON dictionary rather than through page.

    - Switched all code from view into NetworkOutput class (`context_text/export/network_output.py --> NetworkOutput`), got it to work either in view function or stand-alone code with JSON parameters.
    
        - param names exist in two places:
        
            - the forms classes:
            
                - `forms.ArticleSelectForm` - "Select Articles" section of the page.
                - `forms.NetworkOutputForm` - bottom of "Configure Network" section of the page.
                - `forms.PersonSelectForm` - "Select People" section of the page.
                - `forms.RelationSelectionForm` - top of "Configure Network section of the page (things that 
            
            - the parameter name constants-ish (`PARAM_*`) in class `ContextTextBase`.
    
    - Example notebook of JSON invocation: [analysis-network_data_output_example.ipynb](./analysis/analysis-network_data_output_example.ipynb)

#### Add parameters to network data creation

Filtering `Article_Subject`s and `Article_Author`s:

- // `exclude_persons_with_tags_in_list` - list of tags. If Article_Subject or Article_Author is assigned these tags, exclude them from analysis.
- // `include_persons_with_single_word_name` - boolean, defaults to unset/false

Storing rendered network data to database:

- // `database_output` - boolean, defaults to "no".
- // `db_add_timestamp_to_label` - boolean, defaults to "yes".
- // `db_save_data_in_database` - boolean, defaults to "yes". If yes, and if we are outputting to database, stores data in database. If no, does not store in database.
- // `save_data_in_folder` - string path of folder to output the result to. Will include network label in file name if one is set. If empty, will not output network data as file.

To implement each new parameter:

- add `PARAM_*` constant to `ContextTextBase`.
- add fields to appropriate forms

    - `RelationSelectForm` for `exclude_persons_with_tags_in_list` and  `include_persons_with_single_word_name` single setting for both sources and authors, shared across `Article_Data` (ties) and person (nodes) filtering.
    - `NetworkOutputForm` for `database_output`, `db_add_timestamp_to_label`, `db_save_data_in_database`, `save_data_in_folder`.

- Update `NetworkOutput`:

    - _`NetworkOutput.PARAM_NAME_TO_TYPE_MAP`_: Add parameters to `NetworkOutput.PARAM_NAME_TO_TYPE_MAP`
    - _`NetworkOutput.process_network_output_request()`_: if the parameter relates to high-level request processing (saving results to database, saving to a file, etc.), update the method `process_network_output_request()` to retrieve and use the parameter appropriately.
    - _`NetworkOutput.create_query_set()`_: (if needed) update `NetworkOutput.create_query_set()` so it knows of and can process the new parameters.
    - _`NetworkOutput.create_person_dict()`_: update so it takes these parameters into account when it builds person dictionary.

        - when it retrieves `Article_Author`s and `Article_Subject`s, calls `Article_Data` methods `get_article_authors_qs` and `get_quoted_article_sources_qs`, which in turn both call `filter_article_persons`. These are the methods that now can filter to exclude tags, and to exclude single names (based on spaces in `verbatim_name`).

- update `NetworkDataOutput`:

    - so it accepts and stores these parameters in `initialize_from_params()`
    - so `render()` does the right thing when looping over authors and sources.

        - when it retrieves `Article_Author`s and `Article_Subject`s, calls `Article_Data` methods `get_article_authors_qs` and `get_quoted_article_sources_qs`, which in turn both call `filter_article_persons`. These are the methods that now can filter to exclude tags, and to exclude single names (based on spaces in `verbatim_name`).

- if desired, update unit tests to test new parameter.

#### model for network data output requests

Create model to hold network output request parameters JSON, hash of JSON, tags, a few other fields for ease of filtering and querying, and then resulting data, for easy access to results from a given JSON. Then, move all the code from [analysis-network_data_output_example.ipynb](./analysis/analysis-network_data_output_example.ipynb) into methods in NetworkOutput (`process_network_output_request`), to include ability to render network data alongside ability to capture it in model instance/database.

#### unit test(s)

- notebook: [test_data-update_data-network_output.ipynb](./data/test_data/test_data-update_data-network_output.ipynb)

- // update test data:

    - // load existing fixtures.
    - // includes 43 coded with OpenCalais.
    - // make sure we can generate network data from them.
    - single-name data
    
        - there are two single-name sources.

    - things to include in actual data:
        - build out network data for a few different specs (one with single name, one without).

            - specs are in [analysis-network_data_output_example.ipynb](./analysis/analysis-network_data_output_example.ipynb)

        - for each data spec, in NetworkDataOutputLog, capture output from no-single-names for original data and data where set records have single-names introduced, both with details on and details off.
        - also get the hashes and length of output strings you'd expect and store the values in the test case.
        - tag a few Article_Subjects with a random tag ("name_error"...?), for testing removing Article_Subjects that have been assigned tag.

            - Tag `from_press_release` added to the following `Article_Subject` instances:

                - 740 - granholm (person 102)
                - 637 - Mark Meadows (person 224)
                - 677 - Gary Nelund (person 261)

            - Tag `godwin_heights` added to the following `Article_Subject` instances:

                - 623 - Felske, Jon (person 188)
                - 622 - Johnston, Allen E. (person 187)
                - 621 - Hornecker, Kenneth (person 189)

    - remove superuser user from auth.
    - re-export the "export" fixture that includes the network data output log and the tags (should just need those two).

- // make sure existing unit tests work with new data.
- // new unit tests:

    - unit test code is in [analysis-network_data_output_example.ipynb](./analysis/analysis-network_data_output_example.ipynb)
    - simple network data creation test - run with a few specs against test data, make sure I get the right size of output back for each.
    - even lower level, make tests for the method to build person dictionaries, and the base lookup method.
    - also apply a tag to a few article_subject and test for omitting tags? Same as omitting single names (or stacked?).
    - ? - make sure the Article_Data method `filter_article_persons()` works as I intend. To start, create tests in notebook against actual database, using full Article_Subject and Article_Author QuerySets, compare numbers to raw queries. Then, do the same against test database, use numbers to create unit tests. This should be covered by simple network creation tests (they call this method).

### article overview

- notebooks:

    - article overview notebook: [analysis-article_overview.ipynb](./analysis/analysis-article_overview.ipynb)
    - tagging local hard news:

        - [article_coding.ipynb](./data/article_coding/article_coding.ipynb)
        - [newsbank-article_coding.ipynb](./data/article_coding/newsbank-article_coding.ipynb)

- from each paper, date range, and set of tags with counts across each range.
- identify what filtering has already been done:

    - for each paper, sections included, date range, sets of tags.

        - documented in "tagging local hard news" notebooks above.

- Grand Rapids Press local hard news:

    - all articles

        - count: 354,315
        - tags with counts: 

            - tag coded-OpenCalaisV2ArticleCoder: 43816
            - tag export_to_context-20191126-164206: 43816
            - tag grp_month: 441
            - tag local_hard_news: 43816
            - tag minnesota1-20160328: 147
            - tag minnesota2-20160328: 147
            - tag minnesota3-20160328: 147
            - tag prelim_network: 109
            - tag prelim_reliability: 19
            - tag prelim_reliability_combined: 60
            - tag prelim_reliability_test: 60
            - tag prelim_training_001: 40
            - tag prelim_training_002: 30
            - tag prelim_training_003: 60
            - tag prelim_unit_test_001: 10
            - tag prelim_unit_test_002: 10
            - tag prelim_unit_test_003: 10
            - tag prelim_unit_test_004: 10
            - tag prelim_unit_test_005: 10
            - tag prelim_unit_test_006: 10
            - tag prelim_unit_test_007: 10

    - local hard news have tag "`ContextTextBase.TAG_LOCAL_HARD_NEWS`" --> "`local_hard_news`"

        - count: 43,816
        - earliest date: 2005-01-01
        - latest date: 2010-11-30
        - tags with counts:

            - tag coded-OpenCalaisV2ArticleCoder: 43816
            - tag export_to_context-20191126-164206: 43816
            - tag grp_month: 441
            - tag local_hard_news: 43816
            - tag minnesota1-20160328: 147
            - tag minnesota2-20160328: 147
            - tag minnesota3-20160328: 147
            - tag prelim_network: 109
            - tag prelim_reliability: 19
            - tag prelim_reliability_combined: 60
            - tag prelim_reliability_test: 60
            - tag prelim_training_001: 40
            - tag prelim_training_002: 30
            - tag prelim_training_003: 60
            - tag prelim_unit_test_001: 10
            - tag prelim_unit_test_002: 10
            - tag prelim_unit_test_003: 10
            - tag prelim_unit_test_004: 10
            - tag prelim_unit_test_005: 10
            - tag prelim_unit_test_006: 10
            - tag prelim_unit_test_007: 10        

- Detroit News

    - all articles:
    
        - count: 159,716
        - earliest date: 2005-01-01
        - latest date: 2010-10-31
        - tags with counts:

            - tag coded-OpenCalaisV2ArticleCoder: 27
            - tag export_to_context-20191126-164206: 27
            - tag minnesota1-20160409: 27
            - tag minnesota2-20160409: 27
            - tag minnesota3-20160409: 27
            - tag prelim_reliability: 27
            - tag prelim_reliability_combined: 27

    - local hard news:
    
        - author bylines are a mess (partially stored in body of article - I was there, I should have helped them fix this).
        - need to fix those before I can do locally produced hard news.

    - can get counts of articles within hard news sections...?

### Article_Data overview

- notebooks:

    - Article_Data overview notebook: [analysis-article_data_overview.ipynb](./analysis/analysis-article_data_overview.ipynb)
    
- summary for range of GRP that needs to be examined:

    - All 43816 have Article_Data coded by coder_type "OpenCalais_REST_API_v2".
    - date range of related articles:
    
        - min: 2005-01-01
        - max: 2010-11-30

#### missing names

- Look at all Article_Subject with no name information set:

    - first, check if any are related to OpenCalais_REST_API_V2 coding of GRP hard news articles.
    - if so, then we need to try to fix these. (will entail looking at original response from OpenCalais to get verbatim name).
    - and, make sure the future coding sets name columns.

- missing names?

    - are any missing name, verbatim_name, and lookup_name in Article_Subject or Article_Author?
    - Article_Subject:
    
        - there are 55. What to do?
        - first, verify that `verbatim_name` and `lookup_name` are set by default currently.
        
            - updated unit tests to check, fixed `ArticleCoder.lookup_person()` so it always results in those fields being set (checks if they are empty at the end, if so, sets them to full name).
        
        - / then, see if I can use the returned OpenCalais data to populate for those where it is empty.
        
            - OpenCalais JSON is in Article_Data_Notes table.
            - Looks like I had the JSON in a text field, then converted it to a JSON field, and it didn't convert right (or maybe exported it and re-imported it and it didn't reimport right?).
            
                - Need to fix existing data - [analysis-fix_Article_Data_Notes_JSON.ipynb](./analysis/analysis-fix_Article_Data_Notes_JSON.ipynb)
                - Updated coding process so it stores plain JSON text in "content", parses and stores JSON in context_json.
            
            - then, can retrieve JSON for each broken Article_Source and Article_Author and try to see what is going on.
        
        - looks like these are old, from before I added the name, verbatim_name, and lookup_name, and so I just need to fix these 55 manually. New records will get these fields set appropriately in Article_Subject.
        - fixed 53 GRP-only no-names manually (details below in [Appendix - Fixed no-name Article_Subject records](#Appendix---Fixed-no-name-Article_Subject-records):
        
    - Article_Author:
    
        - Verify that authors have "`name`", "`verbatim_name`", and "`lookup_name`" set.
        - check to make sure these things are set in current code
        
            - `ArticleCoder.process_author_string()`
            - `----> ArticleCoder.process_newsbank_grpress_author_string()`
            - `--------> ArticleCoder.process_author_name()`
            - `------------> ArticleCoder.process_person_name()`
            - `----------------> ArticleCoder.lookup_person()`
            - Ah, good, handled in `lookup_person()`. So, probably a result of old code.
            - FUTURE - doh. need to generalize `process_author_string()` - some way to tie to source plus newspaper to appropriate function, or parameterize...?
        
        - added code to fix those that are included in GRP articles into notebook, could be used to fix them all if needed.
        


#### single names

- single names

    - code:
    
        - work notebook: [analysis-omit_single_names.ipynb](./analysis/analysis-omit_single_names.ipynb)
        - notebook where work was done originally (just notes - it was manual): [prelim_month-create_Reliability_Names_data.ipynb](./methods/data_creation/prelim_month-create_Reliability_Names_data.ipynb)
    
- exploration

    - look at names of all people, see how many are single-name.
    
            Processed 85525 people:
            - single_name_count = 3447
            - multi_name_part_count = 82078
            - crazy_name_part_count = 0
    
    - how many persons have only one name part?
    - of those, how many are associated with Article_Subject or Article_Author?
    - compare that to total Article_Subjects with single word verbatim_name.
    - and, see if any match to Article_Subject that doesn't have a single-word verbatim_name.
    - hopefully, single-word verbatim name will be enough to detect single-name, with a few whose two-word names might be a single name...?
    - then, add a single-name tag to all that are single-name, for ease of subsequent filtering?
    - then, look at how many of those single-name people are associated with Article_Subject, Article_Author.
    - code to filter to just single first names is in `context_analysis/views.py --> reliability_names_disagreement_view()`:

            if ( reliability_names_only_first_name == True ):

                # to start, first name needs to not be null and
                #     not be empty.
                reliability_names_qs = reliability_names_qs.filter( 
                    Q( person__first_name__isnull = False ) & ~Q( person__first_name = "" ),
                    Q( person__middle_name__isnull = True ) | Q( person__middle_name = "" ),
                    Q( person__last_name__isnull = True ) | Q( person__last_name = "" ),
                    Q( person__name_prefix__isnull = True ) | Q( person__name_prefix = "" ),
                    Q( person__name_suffix__isnull = True ) | Q( person__name_suffix = "" ),
                    Q( person__nickname__isnull = True ) | Q( person__nickname = "" ),
                )

            #-- END only first name --#

    - need to use a notebook to check how many only have first name, only have middle, only have last, etc., to see what combination we need to detect all people with a single name.
    - then:

        - add "exclude tags" and "exclude single names" to the basic person filter/lookup method.
        - add to the network data creation page and the code that processes that request.
        - Tag all with single name with some tag.

- from Article_Data perspective

    - Article_Subject has name columns that capture the name string from the detection. This is where we should look for single names.

        - some might have at one point matched with a person with one part the same as the single name..., so associated person might not be single-name, but verbatim reference in text was, and the association is incorrect - should check for this, also, and disconnect when this has occurred.

- Article_Subject

        total Article_Subject count = 220076
        automated Article_Subject count = 214229
        automated OpenCalais v.2 Article_Subject count = 212949
        automated GRP Article_Subject count = 212801
        only those with name set - name__isnull = False --> match count = 212801
        single_name records - exclude name__contains = "<space>" --> match count = 13523
        exclude verbatim_name__contains = "<space>" --> match count = 13523
        exclude lookup_name__contains = "<space>" --> match count = 13523
        
        # just sources (quoted)
        total Article_Subject count = 220076
        automated Article_Subject count = 214229
        automated OpenCalais v.2 Article_Subject count = 212949
        automated GRP Article_Subject count = 212801
        automated GRP Article_Subject quoted acount = 114911
        only those with name set - name__isnull = False --> match count = 114911
        single_name records - exclude name__contains = "<space>" --> match count = 1701
        exclude verbatim_name__contains = "<space>" --> match count = 1701
        exclude lookup_name__contains = "<space>" --> match count = 1701

    - compare records with single word (no spaces) `verbatim_name` to the number of name parts for associated person.
    - 


# ====> Appendices

## Appendix - Fixed no-name Article_Subject records

- ID: 5362; AD ID: 1628
- ID: 5369; AD ID: 1630
- ID: 5370; AD ID: 1630
- ID: 5371; AD ID: 1630

Processing 1: 5380 (AS) - Reagan, Nancy ( id = 835; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 550 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/88349d5f-c9f7-3040-ae24-dc957ddaf59a
- ----> found name in JSON: Nancy Reagan ( uuid: http://d.opencalais.com/pershash-1/88349d5f-c9f7-3040-ae24-dc957ddaf59a )
- Links:
    - Article_Subject 5380 ( https://research.local/research/admin/context_text/article_subject/5380/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 835 ( https://research.local/research/admin/context_text/person/835/change/ )

Processing 2: 5381 (AS) - Salahi, Tareq ( id = 836; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 551 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/488a97f4-8635-3048-a149-9281a76e32b0
- ----> found name in JSON: Tareq Salahi ( uuid: http://d.opencalais.com/pershash-1/488a97f4-8635-3048-a149-9281a76e32b0 )
- Links:
    - Article_Subject 5381 ( https://research.local/research/admin/context_text/article_subject/5381/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 836 ( https://research.local/research/admin/context_text/person/836/change/ )

Processing 3: 5382 (AS) - Salahi, Michaele ( id = 837; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 552 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/0f34a475-4c36-3f6e-ba9c-b6b593f2ce09
- ----> found name in JSON: Michaele Salahi ( uuid: http://d.opencalais.com/pershash-1/0f34a475-4c36-3f6e-ba9c-b6b593f2ce09 )
- Links:
    - Article_Subject 5382 ( https://research.local/research/admin/context_text/article_subject/5382/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 837 ( https://research.local/research/admin/context_text/person/837/change/ )

Processing 4: 5383 (AS) - Reagan, Ronald ( id = 838; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 553 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/bffbcc38-baec-3024-bf89-5d00b13c5131
- ----> found name in JSON: Ronald Reagan ( uuid: http://d.opencalais.com/pershash-1/bffbcc38-baec-3024-bf89-5d00b13c5131 )
- Links:
    - Article_Subject 5383 ( https://research.local/research/admin/context_text/article_subject/5383/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 838 ( https://research.local/research/admin/context_text/person/838/change/ )

Processing 5: 5384 (AS) - Ford, Gerald ( id = 839; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?
- 1 - 554 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/0cee64b9-286f-3ba4-90fe-4bf8c90b8ea1
- ----> UUID: http://d.opencalais.com/pershash-1/0cee64b9-286f-3ba4-90fe-4bf8c90b8ea1
- 2 - 4320 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/91b84053-a748-33e1-bfb4-44e22b9e31bc
- ----> UUID: http://d.opencalais.com/pershash-1/91b84053-a748-33e1-bfb4-44e22b9e31bc
- ----> found name in JSON: Gerald R. Ford ( uuid: http://d.opencalais.com/pershash-1/91b84053-a748-33e1-bfb4-44e22b9e31bc )
- Links:

    - Article_Subject 5384 ( https://research.local/research/admin/context_text/article_subject/5384/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 839 ( https://research.local/research/admin/context_text/person/839/change/ )

Processing 6: 5385 (AS) - Portillo, Jose ( id = 840; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 555 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/2dd9a3d6-3a39-3de4-a554-672811aa4ede
- ----> found name in JSON: Jose Lopez Portillo ( uuid: http://d.opencalais.com/pershash-1/2dd9a3d6-3a39-3de4-a554-672811aa4ede )
- Links:
    - Article_Subject 5385 ( https://research.local/research/admin/context_text/article_subject/5385/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 840 ( https://research.local/research/admin/context_text/person/840/change/ )

Processing 7: 5386 (AS) - Haig, Alexander ( id = 841; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 556 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/89c70616-c1f6-30c0-9822-c0e7c99fc05b
- ----> found name in JSON: Alexander Haig ( uuid: http://d.opencalais.com/pershash-1/89c70616-c1f6-30c0-9822-c0e7c99fc05b )
- Links:
    - Article_Subject 5386 ( https://research.local/research/admin/context_text/article_subject/5386/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 841 ( https://research.local/research/admin/context_text/person/841/change/ )

Processing 8: 5387 (AS) - Obama ( id = 842; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 557 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/cbc8c069-5e3f-3778-b9e5-e93f1c6d323a
- ----> found name in JSON: Obama ( uuid: http://d.opencalais.com/pershash-1/cbc8c069-5e3f-3778-b9e5-e93f1c6d323a )
- Links:
    - Article_Subject 5387 ( https://research.local/research/admin/context_text/article_subject/5387/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 842 ( https://research.local/research/admin/context_text/person/842/change/ )

Processing 9: 5388 (AS) - Lloyd, Mike ( id = 341; capture_method = OpenCalais_REST_API_v2 ) (mentioned; individual)
- ----> UUID: 558 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/f88ef4fc-7c2a-3c10-90b2-61f12a35b246
- ----> found name in JSON: Mike Lloyd ( uuid: http://d.opencalais.com/pershash-1/f88ef4fc-7c2a-3c10-90b2-61f12a35b246 )
- Links:
    - Article_Subject 5388 ( https://research.local/research/admin/context_text/article_subject/5388/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 341 ( https://research.local/research/admin/context_text/person/341/change/ )

Processing 10: 5389 (AS) - Kissinger, Henry ( id = 843; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 559 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/0b07dbd9-c12b-38e4-ad03-9bf5c868f5b9
- ----> found name in JSON: Henry Kissinger ( uuid: http://d.opencalais.com/pershash-1/0b07dbd9-c12b-38e4-ad03-9bf5c868f5b9 )
- Links:
    - Article_Subject 5389 ( https://research.local/research/admin/context_text/article_subject/5389/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 843 ( https://research.local/research/admin/context_text/person/843/change/ )

Processing 11: 5390 (AS) - B-Boone, Debby ( id = 759; capture_method = OpenCalais_REST_API ) (quoted; individual)
- ----> UUID: 294 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/846983fe-3fad-3c13-ba50-362c49407b37
- ----> found name in JSON: Debby B-Boone ( uuid: http://d.opencalais.com/pershash-1/846983fe-3fad-3c13-ba50-362c49407b37 )
- Links:
    - Article_Subject 5390 ( https://research.local/research/admin/context_text/article_subject/5390/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 759 ( https://research.local/research/admin/context_text/person/759/change/ )

Processing 12: 5391 (AS) - Trudeau, Pierre ( id = 760; capture_method = OpenCalais_REST_API ) (quoted; individual)
- ----> UUID: 295 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/bf6c1ff3-e4d5-34f9-8c3a-031cbcd9123a
- ----> found name in JSON: Pierre Trudeau ( uuid: http://d.opencalais.com/pershash-1/bf6c1ff3-e4d5-34f9-8c3a-031cbcd9123a )
- Links:
    - Article_Subject 5391 ( https://research.local/research/admin/context_text/article_subject/5391/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 760 ( https://research.local/research/admin/context_text/person/760/change/ )

Processing 13: 5810 (AS) - Holtz, Lou ( id = 966; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 715 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/d177845f-7cb9-30cf-8eec-4261234260ea
- ----> found name in JSON: Lou Holtz ( uuid: http://d.opencalais.com/pershash-1/d177845f-7cb9-30cf-8eec-4261234260ea )
- Links:
    - Article_Subject 5810 ( https://research.local/research/admin/context_text/article_subject/5810/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 966 ( https://research.local/research/admin/context_text/person/966/change/ )

Processing 14: 5811 (AS) - Willingham, Tyrone ( id = 967; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 716 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/d163ad1b-0f5c-3db6-b20e-03d032c3f4a5
- ----> found name in JSON: Tyrone Willingham ( uuid: http://d.opencalais.com/pershash-1/d163ad1b-0f5c-3db6-b20e-03d032c3f4a5 )
- Links:
    - Article_Subject 5811 ( https://research.local/research/admin/context_text/article_subject/5811/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 967 ( https://research.local/research/admin/context_text/person/967/change/ )

Processing 15: 5812 (AS) - Kelly, Brian ( id = 778; capture_method = OpenCalais_REST_API ) (quoted; individual)
- ----> UUID: 467 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/f22dd7ff-c33d-351a-8bfb-b4434f113f9a
- ----> found name in JSON: Brian Kelly ( uuid: http://d.opencalais.com/pershash-1/f22dd7ff-c33d-351a-8bfb-b4434f113f9a )
- Links:
    - Article_Subject 5812 ( https://research.local/research/admin/context_text/article_subject/5812/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 778 ( https://research.local/research/admin/context_text/person/778/change/ )

Processing 16: 5813 (AS) - Beck, Tom ( id = 968; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 717 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/3346f841-f149-35c9-b8fb-ad53a9ccff02
- ----> found name in JSON: Tom Beck ( uuid: http://d.opencalais.com/pershash-1/3346f841-f149-35c9-b8fb-ad53a9ccff02 )
- Links:
    - Article_Subject 5813 ( https://research.local/research/admin/context_text/article_subject/5813/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 968 ( https://research.local/research/admin/context_text/person/968/change/ )

Processing 17: 5814 (AS) - Faust, Gerry ( id = 969; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 718 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/850b27da-a6cc-3ae1-9736-03305c06dfe1
- ----> found name in JSON: Gerry Faust ( uuid: http://d.opencalais.com/pershash-1/850b27da-a6cc-3ae1-9736-03305c06dfe1 )
- Links:
    - Article_Subject 5814 ( https://research.local/research/admin/context_text/article_subject/5814/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 969 ( https://research.local/research/admin/context_text/person/969/change/ )

Processing 18: 5815 (AS) - Weis, Charlie ( id = 970; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 719 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/fe6aefad-e9ec-37e0-b3d0-04e4a487223b
- ----> found name in JSON: Charlie Weis ( uuid: http://d.opencalais.com/pershash-1/fe6aefad-e9ec-37e0-b3d0-04e4a487223b )
- Links:
    - Article_Subject 5815 ( https://research.local/research/admin/context_text/article_subject/5815/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 970 ( https://research.local/research/admin/context_text/person/970/change/ )

Processing 19: 5917 (AS) - Werley, Jennifer ( id = 306; capture_method = None ) (quoted; individual)
- ----> UUID: 232 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/b21f836a-da09-3c62-8e9a-24b761777e92
- ----> found name in JSON: Jennifer Werley ( uuid: http://d.opencalais.com/pershash-1/b21f836a-da09-3c62-8e9a-24b761777e92 )
- Links:
    - Article_Subject 5917 ( https://research.local/research/admin/context_text/article_subject/5917/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 306 ( https://research.local/research/admin/context_text/person/306/change/ )

Processing 20: 5918 (AS) - Howard, Ron ( id = 1009; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 769 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/9d6ed5c1-d841-3032-be3b-7f80500616d6
- ----> found name in JSON: Ron Howard ( uuid: http://d.opencalais.com/pershash-1/9d6ed5c1-d841-3032-be3b-7f80500616d6 )
- Links:
    - Article_Subject 5918 ( https://research.local/research/admin/context_text/article_subject/5918/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 1009 ( https://research.local/research/admin/context_text/person/1009/change/ )

Processing 21: 5919 (AS) - Kimble, Don ( id = 305; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 231 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/82300626-2267-3276-af3d-21aa59f906fc
- ----> found name in JSON: Don Kimble ( uuid: http://d.opencalais.com/pershash-1/82300626-2267-3276-af3d-21aa59f906fc )
- Links:
    - Article_Subject 5919 ( https://research.local/research/admin/context_text/article_subject/5919/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 305 ( https://research.local/research/admin/context_text/person/305/change/ )

Processing 22: 5920 (AS) - Bos, Tom ( id = 303; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 233 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/49e99493-136e-3983-b8a3-9a62b40f2ebc
- ----> found name in JSON: Tom Bos ( uuid: http://d.opencalais.com/pershash-1/49e99493-136e-3983-b8a3-9a62b40f2ebc )
- Links:
    - Article_Subject 5920 ( https://research.local/research/admin/context_text/article_subject/5920/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 303 ( https://research.local/research/admin/context_text/person/303/change/ )

Processing 23: 5921 (AS) - Oppenhuizen, Greg ( id = 304; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 234 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/53e75559-fcd1-33cd-8e1b-5041b4e8c095
- ----> found name in JSON: Greg Oppenhuizen ( uuid: http://d.opencalais.com/pershash-1/53e75559-fcd1-33cd-8e1b-5041b4e8c095 )
- Links:
    - Article_Subject 5921 ( https://research.local/research/admin/context_text/article_subject/5921/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 304 ( https://research.local/research/admin/context_text/person/304/change/ )

Processing 24: 5949 (AS) - Higgins, Rae ( id = 259; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?
- 1 - 34 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/3d8daa6d-310c-3bff-858a-d3b2be449bc5
- ----> UUID: http://d.opencalais.com/pershash-1/3d8daa6d-310c-3bff-858a-d3b2be449bc5
- 2 - 243 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/5303dd0f-6a23-3c14-8bba-f3804264fe14
- ----> UUID: http://d.opencalais.com/pershash-1/5303dd0f-6a23-3c14-8bba-f3804264fe14
- ----> found name in JSON: Rae Higgins ( uuid: http://d.opencalais.com/pershash-1/5303dd0f-6a23-3c14-8bba-f3804264fe14 )
- Links:
    - Article_Subject 5949 ( https://research.local/research/admin/context_text/article_subject/5949/change/ )
    - Article_Data 1757 ( https://research.local/research/admin/context_text/article_data/1757/change/ )
    - Person 259 ( https://research.local/research/admin/context_text/person/259/change/ )

Processing 25: 5950 (AS) - Hoeltzel, Bob ( id = 754; capture_method = OpenCalais_REST_API ) (quoted; individual)
- ----> UUID: 244 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/71489c8f-d301-3fff-b287-8f71a04e319b
- ----> found name in JSON: Bob Hoeltzel ( uuid: http://d.opencalais.com/pershash-1/71489c8f-d301-3fff-b287-8f71a04e319b )
- Links:
    - Article_Subject 5950 ( https://research.local/research/admin/context_text/article_subject/5950/change/ )
    - Article_Data 1757 ( https://research.local/research/admin/context_text/article_data/1757/change/ )
    - Person 754 ( https://research.local/research/admin/context_text/person/754/change/ )

Processing 26: 5951 (AS) - Larsen, Cindy ( id = 263; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 245 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/ff6c6764-07f3-307c-8e23-e3f229b3c6e1
- ----> found name in JSON: Cindy Larsen ( uuid: http://d.opencalais.com/pershash-1/ff6c6764-07f3-307c-8e23-e3f229b3c6e1 )
- Links:
    - Article_Subject 5951 ( https://research.local/research/admin/context_text/article_subject/5951/change/ )
    - Article_Data 1757 ( https://research.local/research/admin/context_text/article_data/1757/change/ )
    - Person 263 ( https://research.local/research/admin/context_text/person/263/change/ )

Processing 27: 5952 (AS) - Nelund, Gary ( id = 261; capture_method = None ) (quoted; individual)
- ----> UUID: 246 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/3dfe7b41-99bc-31a3-8cd9-7af5f5194241
- ----> found name in JSON: Gary Nelund ( uuid: http://d.opencalais.com/pershash-1/3dfe7b41-99bc-31a3-8cd9-7af5f5194241 )
- Links:
    - Article_Subject 5952 ( https://research.local/research/admin/context_text/article_subject/5952/change/ )
    - Article_Data 1757 ( https://research.local/research/admin/context_text/article_data/1757/change/ )
    - Person 261 ( https://research.local/research/admin/context_text/person/261/change/ )

Processing 28: 5975 (AS) - Schumacher, John ( id = 222; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 261 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/ac76f874-4c47-3b7a-891c-35274bec01cf
- ----> found name in JSON: John Schumacher ( uuid: http://d.opencalais.com/pershash-1/ac76f874-4c47-3b7a-891c-35274bec01cf )
- Links:
    - Article_Subject 5975 ( https://research.local/research/admin/context_text/article_subject/5975/change/ )
    - Article_Data 1761 ( https://research.local/research/admin/context_text/article_data/1761/change/ )
    - Person 222 ( https://research.local/research/admin/context_text/person/222/change/ )

Processing 29: 5976 (AS) - Meadows, Mark ( id = 224; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 793 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/e57b2d20-d195-3372-b464-b61dc9685e8a
- ----> found name in JSON: Mark Meadows ( uuid: http://d.opencalais.com/pershash-1/e57b2d20-d195-3372-b464-b61dc9685e8a )
- Links:
    - Article_Subject 5976 ( https://research.local/research/admin/context_text/article_subject/5976/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 224 ( https://research.local/research/admin/context_text/person/224/change/ )

Processing 30: 5977 (AS) - Jansen, Mark ( id = 225; capture_method = OpenCalais_REST_API_v2 ) (mentioned; individual)
- ----> UUID: 713 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/af9c9829-12ec-30b5-907c-295feb5bb43f
- ----> found name in JSON: Mark Jansen ( uuid: http://d.opencalais.com/pershash-1/af9c9829-12ec-30b5-907c-295feb5bb43f )
- Links:
    - Article_Subject 5977 ( https://research.local/research/admin/context_text/article_subject/5977/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 225 ( https://research.local/research/admin/context_text/person/225/change/ )

Processing 31: 5978 (AS) - Parks, Sharon ( id = 228; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 119 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/7eed4ef7-8f42-303c-9379-76bd9dc82617
- ----> found name in JSON: Sharon Parks ( uuid: http://d.opencalais.com/pershash-1/7eed4ef7-8f42-303c-9379-76bd9dc82617 )
- Links:
    - Article_Subject 5978 ( https://research.local/research/admin/context_text/article_subject/5978/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 228 ( https://research.local/research/admin/context_text/person/228/change/ )

Processing 32: 5979 (AS) - Studley, Richard ( id = 227; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 264 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/3dcd6d62-6edd-378c-9475-9369145c4476
- ----> found name in JSON: Richard Studley ( uuid: http://d.opencalais.com/pershash-1/3dcd6d62-6edd-378c-9475-9369145c4476 )
- Links:
    - Article_Subject 5979 ( https://research.local/research/admin/context_text/article_subject/5979/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 227 ( https://research.local/research/admin/context_text/person/227/change/ )

Processing 33: 5980 (AS) - Johnston, Michael ( id = 226; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 262 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/2e1ae8f6-5874-3cc6-bf52-166b8bbe1c2d
- ----> found name in JSON: Michael Johnston ( uuid: http://d.opencalais.com/pershash-1/2e1ae8f6-5874-3cc6-bf52-166b8bbe1c2d )
- Links:
    - Article_Subject 5980 ( https://research.local/research/admin/context_text/article_subject/5980/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 226 ( https://research.local/research/admin/context_text/person/226/change/ )

Processing 34: 5981 (AS) - Owens, Charles ( id = 229; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 263 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/38cdbb87-2ed8-3b43-bc21-15884aca61be
- ----> found name in JSON: Charles Owens ( uuid: http://d.opencalais.com/pershash-1/38cdbb87-2ed8-3b43-bc21-15884aca61be )
- Links:
    - Article_Subject 5981 ( https://research.local/research/admin/context_text/article_subject/5981/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 229 ( https://research.local/research/admin/context_text/person/229/change/ )

Processing 35: 5982 (AS) - Gran-holm, Jennifer ( id = 1029; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 794 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/d52f3bbc-1583-3e2b-a6ac-2f13b113f98d
- ----> found name in JSON: Jennifer Gran-holm ( uuid: http://d.opencalais.com/pershash-1/d52f3bbc-1583-3e2b-a6ac-2f13b113f98d )
- Links:
    - Article_Subject 5982 ( https://research.local/research/admin/context_text/article_subject/5982/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 1029 ( https://research.local/research/admin/context_text/person/1029/change/ )

Processing 36: 5983 (AS) - Jondahl, Lynn ( id = 230; capture_method = OpenCalais_REST_API_v2 ) (mentioned; individual)
- ----> UUID: 795 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/e960018a-7106-3451-a5c0-73c150df113c
- ----> found name in JSON: Lynn Jondahl ( uuid: http://d.opencalais.com/pershash-1/e960018a-7106-3451-a5c0-73c150df113c )
- Links:
    - Article_Subject 5983 ( https://research.local/research/admin/context_text/article_subject/5983/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 230 ( https://research.local/research/admin/context_text/person/230/change/ )

Processing 37: 5984 (AS) - Larson, Chris ( id = 233; capture_method = OpenCalais_REST_API_v2 ) (mentioned; individual)
- ----> UUID: 796 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/c96c46ad-efd9-326a-a6ae-7a30a12bf17a
- ----> found name in JSON: Chris Larson ( uuid: http://d.opencalais.com/pershash-1/c96c46ad-efd9-326a-a6ae-7a30a12bf17a )
- Links:
    - Article_Subject 5984 ( https://research.local/research/admin/context_text/article_subject/5984/change/ )
    - Article_Data 1763 ( https://research.local/research/admin/context_text/article_data/1763/change/ )
    - Person 233 ( https://research.local/research/admin/context_text/person/233/change/ )

Processing 38: 5985 (AS) - Dettloff, Mary ( id = 232; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 265 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/c8de228e-9b09-3a24-9270-b341de103c81
- ----> found name in JSON: Mary Dettloff ( uuid: http://d.opencalais.com/pershash-1/c8de228e-9b09-3a24-9270-b341de103c81 )
- Links:
    - Article_Subject 5985 ( https://research.local/research/admin/context_text/article_subject/5985/change/ )
    - Article_Data 1763 ( https://research.local/research/admin/context_text/article_data/1763/change/ )
    - Person 232 ( https://research.local/research/admin/context_text/person/232/change/ )

Processing 39: 5986 (AS) - Kailing, Pete ( id = 231; capture_method = None ) (quoted; individual)
- ----> UUID: 266 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/c53a7aea-8e4a-386a-80da-108196c43d3f
- ----> found name in JSON: Pete Kailing ( uuid: http://d.opencalais.com/pershash-1/c53a7aea-8e4a-386a-80da-108196c43d3f )
- Links:
    - Article_Subject 5986 ( https://research.local/research/admin/context_text/article_subject/5986/change/ )
    - Article_Data 1763 ( https://research.local/research/admin/context_text/article_data/1763/change/ )
    - Person 231 ( https://research.local/research/admin/context_text/person/231/change/ )

Processing 40: 5996 (AS) - Zimmerman, Matt ( id = 221; capture_method = None ) (mentioned; individual)
- ----> UUID: 808 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/a4fa301c-7b4d-3e24-82cb-7dff64a295b3
- ----> found name in JSON: Matt Zimmerman ( uuid: http://d.opencalais.com/pershash-1/a4fa301c-7b4d-3e24-82cb-7dff64a295b3 )
- Links:
    - Article_Subject 5996 ( https://research.local/research/admin/context_text/article_subject/5996/change/ )
    - Article_Data 1765 ( https://research.local/research/admin/context_text/article_data/1765/change/ )
    - Person 221 ( https://research.local/research/admin/context_text/person/221/change/ )

Processing 41: 5997 (AS) - Swets, Tom ( id = 220; capture_method = None ) (quoted; individual)
- ----> UUID: 267 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/1bfa16b1-8ce7-36db-b778-b63340aa3147
- ----> found name in JSON: Tom Swets ( uuid: http://d.opencalais.com/pershash-1/1bfa16b1-8ce7-36db-b778-b63340aa3147 )
- Links:
    - Article_Subject 5997 ( https://research.local/research/admin/context_text/article_subject/5997/change/ )
    - Article_Data 1765 ( https://research.local/research/admin/context_text/article_data/1765/change/ )
    - Person 220 ( https://research.local/research/admin/context_text/person/220/change/ )

Processing 42: 5998 (AS) - Dean, Bob ( id = 219; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 268 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/330eaf26-ef6d-3300-abf8-bcc68f42c867
- ----> found name in JSON: Bob Dean ( uuid: http://d.opencalais.com/pershash-1/330eaf26-ef6d-3300-abf8-bcc68f42c867 )
- Links:
    - Article_Subject 5998 ( https://research.local/research/admin/context_text/article_subject/5998/change/ )
    - Article_Data 1765 ( https://research.local/research/admin/context_text/article_data/1765/change/ )
    - Person 219 ( https://research.local/research/admin/context_text/person/219/change/ )

Processing 43: 5999 (AS) - Heartwell, George ( id = 218; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 96 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/764c66c9-731a-3480-9f12-466b9fa04c64
- ----> found name in JSON: George Heartwell ( uuid: http://d.opencalais.com/pershash-1/764c66c9-731a-3480-9f12-466b9fa04c64 )
- Links:
    - Article_Subject 5999 ( https://research.local/research/admin/context_text/article_subject/5999/change/ )
    - Article_Data 1765 ( https://research.local/research/admin/context_text/article_data/1765/change/ )
    - Person 218 ( https://research.local/research/admin/context_text/person/218/change/ )

Processing 44: 6011 (AS) - Platschorre, Lee ( id = 1043; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 814 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/6352d7c2-4909-3d35-9f61-93aaf7121cde
- ----> found name in JSON: Lee Ann Platschorre ( uuid: http://d.opencalais.com/pershash-1/6352d7c2-4909-3d35-9f61-93aaf7121cde )
- Links:
    - Article_Subject 6011 ( https://research.local/research/admin/context_text/article_subject/6011/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 1043 ( https://research.local/research/admin/context_text/person/1043/change/ )

Processing 45: 6012 (AS) - Lee, Godfrey ( id = 980; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 736 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/d7252001-7ab4-3ca5-97fe-dec325b595fb
- ----> found name in JSON: Godfrey Lee ( uuid: http://d.opencalais.com/pershash-1/d7252001-7ab4-3ca5-97fe-dec325b595fb )
- Links:
    - Article_Subject 6012 ( https://research.local/research/admin/context_text/article_subject/6012/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 980 ( https://research.local/research/admin/context_text/person/980/change/ )

Processing 46: 6013 (AS) - Hornecker, Kenneth ( id = 189; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 277 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/9db33829-3817-31a7-9a9a-9d421362e922
- ----> found name in JSON: Kenneth Hornecker ( uuid: http://d.opencalais.com/pershash-1/9db33829-3817-31a7-9a9a-9d421362e922 )
- Links:
    - Article_Subject 6013 ( https://research.local/research/admin/context_text/article_subject/6013/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 189 ( https://research.local/research/admin/context_text/person/189/change/ )

Processing 47: 6014 (AS) - Johnston, Allen ( id = 187; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?
- 1 - 275 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/9be64c9c-469f-328c-b56a-65d8447d0509
- ----> UUID: http://d.opencalais.com/pershash-1/9be64c9c-469f-328c-b56a-65d8447d0509
- 2 - 347 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/d7665bbb-1add-37e3-95c0-ff508b507f39
- ----> UUID: http://d.opencalais.com/pershash-1/d7665bbb-1add-37e3-95c0-ff508b507f39
- ----> found name in JSON: Allen E. Johnston ( uuid: http://d.opencalais.com/pershash-1/d7665bbb-1add-37e3-95c0-ff508b507f39 )
- Links:

    - Article_Subject 6014 ( https://research.local/research/admin/context_text/article_subject/6014/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 187 ( https://research.local/research/admin/context_text/person/187/change/ )

Processing 48: 6015 (AS) - VanOeveren, Roderick ( id = 1044; capture_method = OpenCalais_REST_API_v1 ) (mentioned; individual)
- ----> UUID: 815 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API_v1 - uuid: http://d.opencalais.com/pershash-1/b10645fd-2287-3ba6-81dd-a1ece102be03
- ----> found name in JSON: Roderick VanOeveren ( uuid: http://d.opencalais.com/pershash-1/b10645fd-2287-3ba6-81dd-a1ece102be03 )
- Links:
    - Article_Subject 6015 ( https://research.local/research/admin/context_text/article_subject/6015/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 1044 ( https://research.local/research/admin/context_text/person/1044/change/ )

Processing 49: 6016 (AS) - Felske, Jon ( id = 188; capture_method = OpenCalais_REST_API_v2 ) (quoted; individual)
- ----> UUID: 276 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/1aec826d-685f-3f3b-b598-66bd7c1210df
- ----> found name in JSON: Jon Felske ( uuid: http://d.opencalais.com/pershash-1/1aec826d-685f-3f3b-b598-66bd7c1210df )
- Links:
    - Article_Subject 6016 ( https://research.local/research/admin/context_text/article_subject/6016/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 188 ( https://research.local/research/admin/context_text/person/188/change/ )

## Appendix - Fixed no-name Article_Author records


Processing 1: 1663 (AA) - Crawley, Nancy ( id = 350; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 6713 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/d8c9aeb8-97a8-3984-89f8-e59e68d3536e
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 350 - Crawley, Nancy ( Press Business Editor; Press Business Editor ); Article_Data: 1628 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 21189 - Dec 06, 2009, Business ( F1 ), UID: 12C7A9A5AEDA6668 - Vintners soon can toast a tough season ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1663 ( https://research.local/research/admin/context_text/article_author/1663/change/ )
    - Article_Data 1628 ( https://research.local/research/admin/context_text/article_data/1628/change/ )
    - Person 350 ( https://research.local/research/admin/context_text/person/350/change/ )
    - Article 21189 ( https://research.local/research/admin/context_text/article/21189/change/ )

Processing 2: 1665 (AA) - Gustafson, Sven ( id = 758; type = staff; capture_method = OpenCalais_REST_API )
- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 758 - Gustafson, Sven ( Special to the Press; Special to the Press ); Article_Data: 1630 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 21228 - Dec 06, 2009, Business ( F3 ), UID: 12C7A9A5B5DA1648 - When we lose residents, we lose cash - Study says Michigan must attract immigrants, entrepreneurs, knowledge workers ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1665 ( https://research.local/research/admin/context_text/article_author/1665/change/ )
    - Article_Data 1630 ( https://research.local/research/admin/context_text/article_data/1630/change/ )
    - Person 758 ( https://research.local/research/admin/context_text/person/758/change/ )
    - Article 21228 ( https://research.local/research/admin/context_text/article/21228/change/ )

Processing 3: 1669 (AA) - Rademacher, Tom ( id = 340; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 293 - name: OpenCalais API URI (URL) - source: OpenCalais_REST_API - uuid: http://d.opencalais.com/pershash-1/15731878-44be-32b0-91d1-aca841eba232
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 340 - Rademacher, Tom ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1634 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 21172 - Dec 06, 2009, City and Region ( B1 ), UID: 12C7A9A517442D40 - I gave presidential security the slip, too - But this was 30 years ago, and I had a job to do ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1669 ( https://research.local/research/admin/context_text/article_author/1669/change/ )
    - Article_Data 1634 ( https://research.local/research/admin/context_text/article_data/1634/change/ )
    - Person 340 ( https://research.local/research/admin/context_text/person/340/change/ )
    - Article 21172 ( https://research.local/research/admin/context_text/article/21172/change/ )

Processing 4: 1772 (AA) - Mayo, David ( id = 566; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 18801 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/20409113-4046-395a-99bf-b82c1794f1fd
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 566 - Mayo, David ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1731 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 21835 - Dec 11, 2009, Front Page ( A1 ), UID: 12C99FC7DA6D3A70 - Former GVSU coach has what Notre Dame needs - Brian Kelly is the right man for job, even if job is flawed ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1772 ( https://research.local/research/admin/context_text/article_author/1772/change/ )
    - Article_Data 1731 ( https://research.local/research/admin/context_text/article_data/1731/change/ )
    - Person 566 ( https://research.local/research/admin/context_text/person/566/change/ )
    - Article 21835 ( https://research.local/research/admin/context_text/article/21835/change/ )

Processing 5: 1794 (AA) - Chandler, Greg ( id = 302; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 19149 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/7dce3e6c-7e75-3c1b-aa28-b2c94ed81eb5
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 302 - Chandler, Greg ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1752 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28274 - Feb 07, 2010, Lakeshore ( N1 ), UID: 12DC6ADABB51E1F0 - Aquatic center leader to retire - Holland facility director oversaw swimming programs in city for 25 years ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1794 ( https://research.local/research/admin/context_text/article_author/1794/change/ )
    - Article_Data 1752 ( https://research.local/research/admin/context_text/article_data/1752/change/ )
    - Person 302 ( https://research.local/research/admin/context_text/person/302/change/ )
    - Article 28274 ( https://research.local/research/admin/context_text/article/28274/change/ )

Processing 6: 1799 (AA) - McVicar, Brian ( id = 66; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 23925 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/7c6d34db-901a-38ec-8b51-3eb2a313eb36
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 66 - McVicar, Brian ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1757 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28499 - Feb 09, 2010, Business ( A12 ), UID: 12DCB58E30E08708 - Workers' dilemma: move across state or lose jobs ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1799 ( https://research.local/research/admin/context_text/article_author/1799/change/ )
    - Article_Data 1757 ( https://research.local/research/admin/context_text/article_data/1757/change/ )
    - Person 66 ( https://research.local/research/admin/context_text/person/66/change/ )
    - Article 28499 ( https://research.local/research/admin/context_text/article/28499/change/ )

Processing 7: 1804 (AA) - Martinez, Shandra ( id = 36; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 8552 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/b16f4dfb-ee14-3436-8180-5af41fa0cb80
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 36 - Martinez, Shandra ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1761 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28610 - Feb 11, 2010, Business ( A13 ), UID: 12DD5F4EAD3E2740 - Hobby Lobby to move ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1804 ( https://research.local/research/admin/context_text/article_author/1804/change/ )
    - Article_Data 1761 ( https://research.local/research/admin/context_text/article_data/1761/change/ )
    - Person 36 ( https://research.local/research/admin/context_text/person/36/change/ )
    - Article 28610 ( https://research.local/research/admin/context_text/article/28610/change/ )

Processing 8: 1805 (AA) - Luke, Peter ( id = 223; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 5335 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/ba14df73-e6b5-3aa8-8c49-b5883dbb609f
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 223 - Luke, Peter ( Grand Rapids Press Lansing Bureau; Grand Rapids Press Lansing Bureau ); Article_Data: 1762 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28598 - Feb 11, 2010, City and Region ( A3 ), UID: 12DD5F4EA4CE3DC0 - Governor endorses tax on services - Granholm's proposal resembles CEO-backed plan ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1805 ( https://research.local/research/admin/context_text/article_author/1805/change/ )
    - Article_Data 1762 ( https://research.local/research/admin/context_text/article_data/1762/change/ )
    - Person 223 ( https://research.local/research/admin/context_text/person/223/change/ )
    - Article 28598 ( https://research.local/research/admin/context_text/article/28598/change/ )

Processing 9: 1806 (AA) - McVicar, Brian ( id = 66; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 23925 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/7c6d34db-901a-38ec-8b51-3eb2a313eb36
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 66 - McVicar, Brian ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1763 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28649 - Feb 11, 2010, City and Region ( A6 ), UID: 12DD5F4ED966B368 - Report of 'cougar' just a big kitty - State officials say paw prints don't measure up ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1806 ( https://research.local/research/admin/context_text/article_author/1806/change/ )
    - Article_Data 1763 ( https://research.local/research/admin/context_text/article_data/1763/change/ )
    - Person 66 ( https://research.local/research/admin/context_text/person/66/change/ )
    - Article 28649 ( https://research.local/research/admin/context_text/article/28649/change/ )

Processing 10: 1808 (AA) - Harger, Jim ( id = 217; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 11378 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/e41fbafe-9305-35b2-8a9a-efb6fa466f26
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 217 - Harger, Jim ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1765 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28741 - Feb 12, 2010, City and Region ( A3 ), UID: 12DE66A5CD41FF48 - Mayor spreads message of unity - George Heartwell pushes for consolidating regional government services ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1808 ( https://research.local/research/admin/context_text/article_author/1808/change/ )
    - Article_Data 1765 ( https://research.local/research/admin/context_text/article_data/1765/change/ )
    - Person 217 ( https://research.local/research/admin/context_text/person/217/change/ )
    - Article 28741 ( https://research.local/research/admin/context_text/article/28741/change/ )

Processing 11: 1811 (AA) - VandeBunte, Matt ( id = 591; type = staff; capture_method = OpenCalais_REST_API_v2 )
- ----> UUID: 34972 - name: OpenCalais API V2 URI (URL) - source: OpenCalais_REST_API_v2 - uuid: http://d.opencalais.com/pershash-1/1f8f7a52-f04c-3320-94b8-20c32a117db6
- !!!! ERROR: Person NOT FOUND in JSON ( Person: 591 - VandeBunte, Matt ( The Grand Rapids Press; The Grand Rapids Press ); Article_Data: 1768 - automated ( ADCT: OpenCalais_REST_API_v2 )  -- Article: 28846 - Feb 13, 2010, City and Region ( A3 ), UID: 12DE66A655DC4870 - Godwin board seeks options on campus tour - Budget woes force district to consider closing building ( Grand Rapids Press, The ).
- Links:

    - Article_Author 1811 ( https://research.local/research/admin/context_text/article_author/1811/change/ )
    - Article_Data 1768 ( https://research.local/research/admin/context_text/article_data/1768/change/ )
    - Person 591 ( https://research.local/research/admin/context_text/person/591/change/ )
    - Article 28846 ( https://research.local/research/admin/context_text/article/28846/change/ )

